In [2]:
!pip uninstall -y mujoco-py
!pip install mujoco-py==2.1.2.14
!pip install tensorflow
!pip install torch
!pip install gymnasium

Found existing installation: mujoco-py 2.1.2.14
Uninstalling mujoco-py-2.1.2.14:
  Successfully uninstalled mujoco-py-2.1.2.14
  Using cached mujoco_py-2.1.2.14-py3-none-any.whl.metadata (669 bytes)
Using cached mujoco_py-2.1.2.14-py3-none-any.whl (2.4 MB)


In [3]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import pickle
import imageio
from IPython.display import Video

def run(episodes, is_training=True, render=False, alpha=0.9, gamma=0.9, epsilon_decay=0.0001):
    # Initialisation de l'environnement avec option de visualisation
    env = gym.make('Taxi-v3', render_mode='rgb_array' if render else None)

    # Initialisation de la Q-table
    if is_training:
        q = np.zeros((env.observation_space.n, env.action_space.n))
    else:
        try:
            with open('taxi.pkl', 'rb') as f:
                q = pickle.load(f)
        except FileNotFoundError:
            print("Erreur : le fichier 'taxi.pkl' n'a pas été trouvé. Veuillez entraîner le modèle d'abord.")
            return

    # Paramètres d'exploration
    epsilon = 1.0
    rng = np.random.default_rng()

    # Pour suivre les récompenses par épisode
    rewards_per_episode = np.zeros(episodes)
    frames = []  # Pour enregistrer les frames si on rend

    for i in range(episodes):
        state = env.reset()[0]  # Réinitialisation de l'état
        terminated, truncated = False, False
        total_rewards = 0  # Récompenses cumulées pour cet épisode

        while not terminated and not truncated:
            # Sélection de l'action (exploration vs exploitation)
            if is_training and rng.random() < epsilon:
                action = env.action_space.sample()  # Action aléatoire
            else:
                action = np.argmax(q[state, :])  # Action optimale basée sur Q-table

            # Exécuter l'action et obtenir les résultats
            new_state, reward, terminated, truncated, _ = env.step(action)
            total_rewards += reward

            # Mettre à jour la Q-table si on est en mode entraînement
            if is_training:
                q[state, action] += alpha * (reward + gamma * np.max(q[new_state, :]) - q[state, action])

            state = new_state  # Mettre à jour l'état

            # Enregistrer les frames si on rend
            if render:
                frames.append(env.render())

        # Mettre à jour epsilon (réduction progressive de l'exploration)
        epsilon = max(epsilon - epsilon_decay, 0)
        rewards_per_episode[i] = total_rewards  # Stocker les récompenses pour cet épisode

        # Afficher le progrès toutes les 1000 itérations
        if is_training and i % 1000 == 0:
            print(f"Épisode {i}, Récompense moyenne (100 derniers épisodes) : {np.mean(rewards_per_episode[max(0, i-100):i])}")

    env.close()

    # Sauvegarde de la Q-table après entraînement
    if is_training:
        with open("taxi.pkl", "wb") as f:
            pickle.dump(q, f)
        print("Entraînement terminé et Q-table sauvegardée dans 'taxi.pkl'.")
    else:
        # Générer une vidéo si en mode rendu
        if render:
            video_path = "taxi_simulation.mp4"
            imageio.mimsave(video_path, frames, fps=15)
            print(f"Vidéo sauvegardée à : {video_path}")
            return video_path

# Entraîner l'agent
print("Début de l'entraînement...")
run(15000, alpha=0.9, gamma=0.9, epsilon_decay=0.0001)

# Tester l'agent avec visualisation et enregistrement vidéo
print("Début du test...")
video_path = run(5, is_training=False, render=True)

# Afficher la vidéo
if video_path:
    Video(video_path, embed=True)

# Télécharger la vidéo
from google.colab import files
files.download(video_path)


Début de l'entraînement...
Épisode 0, Récompense moyenne (100 derniers épisodes) : nan
Épisode 1000, Récompense moyenne (100 derniers épisodes) : -593.0
Épisode 2000, Récompense moyenne (100 derniers épisodes) : -269.28
Épisode 3000, Récompense moyenne (100 derniers épisodes) : -143.23
Épisode 4000, Récompense moyenne (100 derniers épisodes) : -75.08
Épisode 5000, Récompense moyenne (100 derniers épisodes) : -47.81
Épisode 6000, Récompense moyenne (100 derniers épisodes) : -30.1
Épisode 7000, Récompense moyenne (100 derniers épisodes) : -14.98
Épisode 8000, Récompense moyenne (100 derniers épisodes) : -6.28
Épisode 9000, Récompense moyenne (100 derniers épisodes) : 2.85
Épisode 10000, Récompense moyenne (100 derniers épisodes) : 7.96
Épisode 11000, Récompense moyenne (100 derniers épisodes) : 7.52
Épisode 12000, Récompense moyenne (100 derniers épisodes) : 7.94
Épisode 13000, Récompense moyenne (100 derniers épisodes) : 7.81
Épisode 14000, Récompense moyenne (100 derniers épisodes) : 7

Vidéo sauvegardée à : taxi_simulation.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>